# Hybrid Quantum-Classical Neural Networks

## Training with PyTorch and PennyLane

Hybrid quantum-classical neural networks combine the power of classical deep learning with quantum circuits. The quantum layer can learn patterns that are difficult for classical networks.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Note: PennyLane would be used for actual quantum circuits
# For this demonstration, we simulate the quantum layer

print(f"PyTorch version: {torch.__version__}")

## The Hybrid Architecture

A hybrid quantum-classical neural network has:

1. **Classical pre-processing**: Classical neural network layers
2. **Quantum layer**: Parameterized quantum circuit
3. **Classical post-processing**: Final classical layers

```
Input → [Classical Layers] → [Quantum Layer] → [Classical Layers] → Output
```

In [ ]:
class SimulatedQuantumLayer(nn.Module):
    """
    A simulated quantum layer for demonstration.

    In practice, this would be a PennyLane quantum circuit.
    """
    def __init__(self, n_qubits, n_layers=2):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers

        # Quantum circuit parameters (rotation angles)
        n_params = n_qubits * n_layers * 3  # 3 rotation angles per qubit per layer
        self.quantum_params = nn.Parameter(torch.randn(n_params) * 0.1)

    def forward(self, x):
        """
        Simulate quantum circuit processing.

        In a real implementation, this would:
        1. Encode input data into quantum states
        2. Apply parameterized quantum gates
        3. Measure expectation values
        """
        batch_size = x.shape[0]

        # Simulate quantum processing with a non-linear transformation
        # This mimics the behavior of a variational quantum circuit

        # Reshape parameters for processing
        params = self.quantum_params.view(self.n_layers, self.n_qubits, 3)

        # Apply simulated rotations (simplified)
        output = torch.zeros(batch_size, self.n_qubits)

        for i in range(self.n_qubits):
            # Simulate expectation value measurement
            # In reality, this would be <Z_i> after the quantum circuit
            angle_sum = torch.sum(torch.abs(params[:, i, :]))
            output[:, i] = torch.tanh(x[:, i % x.shape[1]] + angle_sum)

        return output

class HybridQuantumNeuralNetwork(nn.Module):
    """
    A hybrid quantum-classical neural network.
    """
    def __init__(self, input_size, hidden_size, n_qubits, output_size):
        super().__init__()

        # Classical pre-processing
        self.classical_pre = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_qubits),
            nn.Tanh()  # Normalize to [-1, 1] for quantum encoding
        )

        # Quantum layer
        self.quantum_layer = SimulatedQuantumLayer(n_qubits)

        # Classical post-processing
        self.classical_post = nn.Sequential(
            nn.Linear(n_qubits, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        x = self.classical_pre(x)
        x = self.quantum_layer(x)
        x = self.classical_post(x)
        return x

# Create model
model = HybridQuantumNeuralNetwork(
    input_size=4,
    hidden_size=16,
    n_qubits=4,
    output_size=2
)

print("Hybrid Quantum Neural Network Architecture:")
print(model)


## Create a Simple Dataset

In [ ]:
def create_dataset(n_samples=500, noise=0.1):
    """
    Create a simple binary classification dataset.
    """
    np.random.seed(42)
    
    # Two concentric circles (like sklearn's make_circles)
    n_per_class = n_samples // 2
    
    # Class 0: inner circle
    r0 = np.random.normal(0.3, noise, n_per_class)
    theta0 = np.random.uniform(0, 2*np.pi, n_per_class)
    X0 = np.column_stack([r0 * np.cos(theta0), r0 * np.sin(theta0)])
    
    # Class 1: outer circle
    r1 = np.random.normal(0.7, noise, n_per_class)
    theta1 = np.random.uniform(0, 2*np.pi, n_per_class)
    X1 = np.column_stack([r1 * np.cos(theta1), r1 * np.sin(theta1)])
    
    # Combine
    X = np.vstack([X0, X1])
    y = np.hstack([np.zeros(n_per_class), np.ones(n_per_class)])
    
    # Add extra features (to match input_size=4)
    X = np.column_stack([X, np.random.randn(n_samples, 2) * 0.1])
    
    # Shuffle
    idx = np.random.permutation(n_samples)
    X, y = X[idx], y[idx]
    
    return X.astype(np.float32), y.astype(np.int64)

# Create dataset
X, y = create_dataset()

# Visualize
plt.figure(figsize=(8, 6))
plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', label='Class 0', alpha=0.6)
plt.scatter(X[y==1, 0], X[y==1, 1], c='red', label='Class 1', alpha=0.6)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Binary Classification Dataset')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Dataset shape: X={X.shape}, y={y.shape}")

## Training the Hybrid Model

In [ ]:
# Convert to PyTorch tensors
X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)

# Split into train/test
n_train = int(0.8 * len(X))
X_train, X_test = X_tensor[:n_train], X_tensor[n_train:]
y_train, y_test = y_tensor[:n_train], y_tensor[n_train:]

# Create data loaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize model, loss, and optimizer
model = HybridQuantumNeuralNetwork(
    input_size=4,
    hidden_size=16,
    n_qubits=4,
    output_size=2
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
n_epochs = 50
train_losses = []
train_accuracies = []

print("Training Hybrid Quantum Neural Network...")
print("=" * 50)

for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0
    
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
    
    avg_loss = epoch_loss / len(train_loader)
    accuracy = correct / total
    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}: Loss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}")

print("\nTraining complete!")

## Visualizing Training Progress

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss
axes[0].plot(train_losses, 'b-', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(train_accuracies, 'g-', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training Accuracy')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Evaluate on Test Set

In [ ]:
# Evaluate
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    
print(f"Test Accuracy: {accuracy:.4f}")

# Visualize decision boundary
def plot_decision_boundary(model, X, y):
    """
    Plot the decision boundary of the classifier.
    """
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    
    # Create grid points
    grid = np.c_[xx.ravel(), yy.ravel(), 
                 np.zeros(xx.ravel().shape), 
                 np.zeros(xx.ravel().shape)]
    grid_tensor = torch.from_numpy(grid.astype(np.float32))
    
    model.eval()
    with torch.no_grad():
        outputs = model(grid_tensor)
        probs = torch.softmax(outputs, dim=1)[:, 1].numpy()
    
    probs = probs.reshape(xx.shape)
    
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, probs, levels=20, cmap='RdBu', alpha=0.8)
    plt.colorbar(label='Probability of Class 1')
    plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', label='Class 0', edgecolors='white')
    plt.scatter(X[y==1, 0], X[y==1, 1], c='red', label='Class 1', edgecolors='white')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title('Decision Boundary')
    plt.legend()
    plt.show()

plot_decision_boundary(model, X, y)

## Comparing with Classical-Only Network

In [ ]:
class ClassicalNeuralNetwork(nn.Module):
    """
    A purely classical neural network for comparison.
    """
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )
        
    def forward(self, x):
        return self.network(x)

# Train classical model
classical_model = ClassicalNeuralNetwork(input_size=4, hidden_size=16, output_size=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classical_model.parameters(), lr=0.01)

print("Training Classical Neural Network...")
print("=" * 50)

classical_losses = []
for epoch in range(n_epochs):
    classical_model.train()
    epoch_loss = 0
    
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = classical_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    classical_losses.append(epoch_loss / len(train_loader))
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}: Loss = {classical_losses[-1]:.4f}")

# Evaluate
classical_model.eval()
with torch.no_grad():
    outputs = classical_model(X_test)
    _, predicted = torch.max(outputs, 1)
    classical_accuracy = (predicted == y_test).sum().item() / len(y_test)

print(f"\nClassical Model Test Accuracy: {classical_accuracy:.4f}")
print(f"Hybrid Model Test Accuracy: {accuracy:.4f}")

## Summary

**Key Takeaways:**

1. **Hybrid architecture** combines classical and quantum layers
2. **Quantum layers** can be trained with backpropagation (parameter-shift rule)
3. **Gradients flow** through the quantum circuit just like classical layers
4. **Potential advantages**: Quantum layers may learn different patterns

**Practical Considerations:**
- Real quantum hardware introduces noise and errors
- Shot noise (finite samples) affects gradient estimation
- Barren plateaus can be a challenge in quantum neural networks
- Current advantage is mainly for specific quantum-inspired problems